RNN numba package
=================



## A few settings



### autoreload



In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

### project root



In [1]:
import os
import sys
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Get parent directory (= the project root)
project_root = os.path.join(current_dir, '..')
# Append to system path
sys.path.append(project_root)
print(sys.path)

print("Python exe")
print(sys.executable)

Python exe
/home/leon/mambaforge/envs/dual_data/bin/python

### imports and setting up matplotlib



In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle as pkl

sns.set_context("poster")
sns.set_style("ticks")
plt.rc("axes.spines", top=False, right=False)
fig_path = '../figs/perf'
golden_ratio = (5**.5 - 1) / 2
width = 6
height = width * golden_ratio

matplotlib.rcParams['figure.figsize'] = [width, height]

matplotlib.rcParams['lines.markersize'] = 5
%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [1]:
print('a test figure')
plt.figure()
plt.plot([1,2,3,4], '-o')
plt.xlabel('x')
# plt.show()

plt.figure()
plt.plot([1,2,3,4], '-o')
plt.xlabel('x')
# plt.show()

1+1

a test figure
2

## Continuous rate model



### imports



In [1]:
from src.model.rate_model import Network

### Single trial



To run a simulation, first we need to define a network model.
The class Network takes two arguments:

1.  the name of the configuration file that defines the model;
2.  the name of the output file that will contain the simulation data.



In [1]:
model = Network('config_bump.yml', 'test')

Loading config from /home/leon/models/rnn_numba/conf/config_bump.yml
Saving to /home/leon/models/rnn_numba/data/simul/test.h5

Then one just runs the model with



In [1]:
model.run()

Generating matrix Cij
Saving matrix to /home/leon/models/rnn_numba/data/matrix/Cij.npy
Running simulation
Elapsed (with compilation) = 6.908852840075269s